In [1]:
! pip install openai-agents
! pip install resend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.3/223.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel
import resend
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

secret_value_0 = user_secrets.get_secret("DEEPSEEK_API_KEY")
secret_value_1 = user_secrets.get_secret("GEMINI_API_KEY")
secret_value_2 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_3 = user_secrets.get_secret("OPENAI_API_KEY")
secret_value_4 = user_secrets.get_secret("RESEND_API_KEY")

os.environ["DEEPSEEK_API_KEY"] = secret_value_0
os.environ["GEMINI_API_KEY"] = secret_value_1
os.environ["GROQ_API_KEY"] = secret_value_2
os.environ["OPENAI_API_KEY"] = secret_value_3
os.environ["RESEND_API_KEY"] = secret_value_4
resend.api_key = secret_value_4

In [3]:
instructions1 = "You are a sales agent working for FinSight, \
a company that provides an AI-powered platform for real-time financial insights and forecasting. \
You write professional, serious cold emails aimed at CFOs, finance managers, and decision-makers, emphasizing credibility, value, and trust."

instructions2 = "You are a witty, engaging sales agent working for FinSight, \
a company providing AI-driven financial insights and forecasting. \
You write clever, attention-grabbing cold emails that appeal to CFOs and finance leaders, balancing humor with clear business value to encourage responses."

instructions3 = "You are a busy, results-focused sales agent working for FinSight, \
offering AI-based financial forecasting tools. \
You write concise, to-the-point cold emails targeting finance executives, highlighting key benefits quickly and effectively to maximize engagement."


In [4]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://openrouter.ai/api/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"



deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=secret_value_0)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=secret_value_1)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=secret_value_2)

deepseek_model=OpenAIChatCompletionsModel(model="deepseek/deepseek-chat-v3-0324:free",openai_client=deepseek_client)
gemini_model=OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=gemini_client)
groq_model=OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile",openai_client=groq_client)


sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Groq Sales Agent",instructions=instructions3,model=groq_model)

description = "Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [5]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    params = {"from": "onboarding@resend.dev","to": ["evil32894@gmail.com"],"subject": subject,"html": html_body}
    response = resend.Emails.send(params)
    return {"status": "success", "response": response}


In [6]:
subject_instructions = "You are an expert email copywriter. " \
"Given the main body of a cold sales email, craft a subject line that is concise, compelling, and optimized for high open rates. " \
"Focus on clarity, engagement, and relevance to the recipient's interests or pain points."

html_instructions = "You are an expert email designer. " \
"Convert a plain text or markdown email body into a clean, well-formatted HTML email. " \
"Ensure the layout is visually appealing, readable, and highlights key points clearly while maintaining professional style."

subject_writer=Agent(name="Email Subject Writer",instructions=subject_instructions,model="gpt-4o-mini")
subject_tool=subject_writer.as_tool(tool_name="subject_writer",tool_description="Generate a concise, compelling subject line for a cold sales email")

html_converter = Agent(name="HTML Email Body Converter",instructions=html_instructions,model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a plain text or markdown email body into a professional HTML email")

In [7]:
email_tools=[subject_tool,html_tool,send_html_email]

In [8]:
instructions = """
You are the Email Manager. Your task is to take a plain text email body and send it professionally. 
Follow these steps in order:

1. Use the subject_writer tool to generate a compelling subject line that maximizes open rates.
2. Use the html_converter tool to convert the plain text email body into a clean, well-formatted HTML email.
3. Use the send_html_email tool to send the email using the generated subject and HTML body.

Do not skip any step. Ensure the email is clear, engaging, and professional.
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Format the email into HTML and send it to the recipient"
)


tools = [tool1, tool2, tool3]


handoffs = [emailer_agent]

In [9]:
sales_manager_instructions = """
You are the Sales Manager at ComplAI. Your mission is to generate and send the single most effective cold sales email using the sales_agent tools.

Workflow:
1. Generate Drafts: Call all three sales_agent tools to create three unique cold email drafts. Do not skip any tool and do not write drafts yourself.
2. Evaluate and Select: Critically assess all drafts. Pick the one email that is most likely to get a positive response based on clarity, engagement, and persuasiveness. You may regenerate drafts if needed.
3. Handoff for Sending: Send ONLY the selected draft to the 'Email Manager' agent. The Email Manager will handle formatting and sending.

Rules:
- Drafts must come exclusively from the sales_agent tools.
- Never modify drafts yourself; only select the best one.
- Ensure the send_email or send_html_email tool is used exactly once via the Email Manager.
- Only one email draft should be handed off; do not send multiple emails.
"""

sales_manager = Agent(name="Sales Manager",instructions=sales_manager_instructions,
                      tools=tools,handoffs=handoffs,model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Smart Sales Email Pipeline_HELLO"):
    result = await Runner.run(sales_manager, message,max_turns=50)


In [10]:
from pydantic import BaseModel
from typing import List
from agents import Agent, Runner, trace, input_guardrail, GuardrailFunctionOutput

class CleanMessageOutput(BaseModel):
    cleaned_message: str
    removed_names: List[str]
    removed_symbols: List[str]

guardrail_agent = Agent(
    name="Name & Symbol Cleaner",
    instructions="""
    Detect any personal names in the message. Remove them.
    Also detect and remove copyright symbols like ©, ®, ™.
    Return the cleaned message along with lists of removed names and symbols.
    """,
    output_type=CleanMessageOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def clean_message_guardrail(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context or {})
    cleaned_text = result.final_output.cleaned_message
    removed_names = result.final_output.removed_names
    removed_symbols = result.final_output.removed_symbols
    ctx.context = ctx.context or {}
    ctx.context["cleaned_message"] = cleaned_text
    return GuardrailFunctionOutput(
        output_info={
            "cleaned_message": cleaned_text,
            "removed_names": removed_names,
            "removed_symbols": removed_symbols
        },
        tripwire_triggered=False
    )

careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[clean_message_guardrail]
)

message = "Send out a cold sales email addressed to Dear CEO from Alice © 2025"

with trace("Protected Automated SDR with Cleaning_HELLO"):
    result = await Runner.run(
        careful_sales_manager,
        message,
        context={},
        max_turns=100
    )
